In [2]:
# --- 1️⃣ Setup & Install Dependencies
!pip install implicit==0.7.2 tqdm --quiet
!pip install implicit==0.7.2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 72.7 MB/s eta 0:00:00:00:0100:01


In [4]:
# --- 2️⃣ Import Libraries
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tqdm import tqdm
import implicit
import torch

In [5]:
# --- 3️⃣ Cek GPU Availability
use_gpu = torch.cuda.is_available()
print(f"🔍 GPU detected: {use_gpu}")
if use_gpu:
    print("✅ GPU mode aktif (CUDA T4)")
else:
    print("⚙️ CPU mode aktif)")

# --- 4️⃣ Load Data
train = pd.read_csv("/kaggle/input/data-train-book/train.csv")
print("Data sample:")
display(train.head())

# --- 5️⃣ Encode user dan item
user_map = {u: i for i, u in enumerate(train['user_id'].unique())}
item_map = {i: j for j, i in enumerate(train['item_id'].unique())}

train['user_idx'] = train['user_id'].map(user_map)
train['item_idx'] = train['item_id'].map(item_map)


🔍 GPU detected: True
✅ GPU mode aktif (CUDA T4)
Data sample:


,user_id,item_id
0,8,0002005018
1,8,074322678X
2,8,0887841740
3,8,1552041778
4,8,1567407781


In [6]:
# --- 6️⃣ Split Train / Validation

# Fungsi aman untuk sampling per-user
def safe_sample(x, frac=0.8):
    if len(x) == 1:
        return x  # jangan buang kalau cuma 1 interaksi
    else:
        return x.sample(frac=frac, random_state=42)

# Terapkan ke seluruh user
train_data = train.groupby('user_idx', group_keys=False).apply(safe_sample)
val_data = train[~train.index.isin(train_data.index)]

print("📊 Jumlah data:")
print(f"  Total data asli : {len(train):,}")
print(f"  Train data       : {len(train_data):,}")
print(f"  Validation data  : {len(val_data):,}")

# Pastikan jumlah user & item tetap sama
n_users = len(user_map)
n_items = len(item_map)

# Buat matriks sparse
train_mat = sparse.csr_matrix(
    (np.ones(len(train_data)), (train_data['user_idx'], train_data['item_idx'])),
    shape=(n_users, n_items)
)
val_mat = sparse.csr_matrix(
    (np.ones(len(val_data)), (val_data['user_idx'], val_data['item_idx'])),
    shape=(n_users, n_items)
)

print(f"\n✅ Matriks Train: {train_mat.shape[0]} users × {train_mat.shape[1]} items")
print(f"✅ Matriks Val  : {val_mat.shape[0]} users × {val_mat.shape[1]} items")


📊 Jumlah data:
  Total data asli : 269,764
  Train data       : 215,489
  Validation data  : 54,275

✅ Matriks Train: 13876 users × 123069 items
✅ Matriks Val  : 13876 users × 123069 items


/tmp/ipykernel_37/1086856176.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_data = train.groupby('user_idx', group_keys=False).apply(safe_sample)


In [7]:
# --- 7️⃣ Fungsi Evaluasi MAP@K ---
def mapk(actual, predicted, k=10):
    """Mean Average Precision at K"""
    scores = []
    for a, p in zip(actual, predicted):
        if len(a) == 0:
            scores.append(0)
        else:
            relevance = [1 if i in a else 0 for i in p[:k]]
            if sum(relevance) == 0:
                scores.append(0)
            else:
                precisions = [
                    np.mean(relevance[:i+1]) for i, r in enumerate(relevance) if r == 1
                ]
                scores.append(np.mean(precisions))
    return np.mean(scores)

In [8]:
# 🔷 VERSI  IMPROVED IBCF — Cosine + BM25 + Cold-Start Handling
import warnings
warnings.filterwarnings("ignore")

from implicit.nearest_neighbours import CosineRecommender, bm25_weight
from tqdm import tqdm
import numpy as np
import pandas as pd

print("\n🚀 Training Improved IBCF (Cosine + BM25) ...")

# --- 1️⃣ BM25 weighting ---
train_weighted = bm25_weight(train_mat.T, K1=100, B=0.8).T.tocsr()

# --- 2️⃣ Fit model ---
ibcf_model = CosineRecommender(K=100)
ibcf_model.fit(train_weighted.T)

# --- 3️⃣ Generate rekomendasi ---
ibcf_preds = []
top_n = 10

for u in tqdm(range(n_users)):
    # Rekomendasi untuk user u
    recs, _ = ibcf_model.recommend(
        userid=u,
        user_items=train_mat,             # harus users × items
        N=top_n,
        filter_already_liked_items=True
    )

    # Jika kosong (cold-start), fallback pakai item populer
    if len(recs) == 0:
        recs, _ = ibcf_model.similar_items(0, N=top_n)

    ibcf_preds.append(recs)

# --- 4️⃣ Evaluasi MAP@10 ---
actual = [val_data[val_data['user_idx'] == u]['item_idx'].tolist() for u in range(n_users)]

def mapk(actual, predicted, k=10):
    scores = []
    for a, p in zip(actual, predicted):
        if len(a) == 0:
            scores.append(0)
        else:
            relevance = [1 if i in a else 0 for i in p[:k]]
            precisions = [np.mean(relevance[:i+1]) for i, r in enumerate(relevance) if r == 1]
            scores.append(np.mean(precisions) if len(precisions) > 0 else 0)
    return np.mean(scores)

map10 = mapk(actual, ibcf_preds, k=10)
print(f"\n📈 MAP@10 : {map10:.4f}")

# --- 5️⃣ Simpan hasil ke CSV ---
rec_df = pd.DataFrame({
    "user_id": [list(user_map.keys())[u] for u in range(n_users)],
    **{f"item_{i+1}": [list(item_map.keys())[r[i]] if i < len(r) else None
                        for r in ibcf_preds] for i in range(10)}
})
rec_df.to_csv("/kaggle/working/ibcf_top10_fixed.csv", index=False)
print("✅ Hasil rekomendasi disimpan ke '/kaggle/working/ibcf_top10_fixed.csv'")

# --- 6️⃣ Contoh 5 user pertama ---
display(rec_df.head())



🚀 Training Improved IBCF (Cosine + BM25) ...


  0%|          | 0/13876 [00:00<?, ?it/s]

100%|██████████| 13876/13876 [00:00<00:00, 149967.95it/s]



📈 MAP@10 : 0.0005
✅ Hasil rekomendasi disimpan ke '/kaggle/working/ibcf_top10_fixed.csv'


,user_id,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10
0,8,0553573926,0446515671,0553578936,0553801309,000649840X,0198320264,0553578774,0060083263,0316119598,0821773682
1,99,0553573926,0446515671,0553578936,0553801309,000649840X,0198320264,0553578774,0060083263,0316119598,0821773682
2,114,0553573926,0446515671,0553578936,0553801309,000649840X,0198320264,0553578774,0060083263,0316119598,0821773682
3,160,0553573926,0446515671,0553578936,0553801309,000649840X,0198320264,0553578774,0060083263,0316119598,0821773682
4,183,0553573926,0446515671,0553578936,0553801309,000649840X,0198320264,0553578774,0060083263,0316119598,0821773682


In [9]:
# 🔷 VERSI 1 — MODEL-BASED RECOMMENDER (ALS Optimized & Fixed + MAP@10)
from implicit.als import AlternatingLeastSquares
from tqdm import tqdm
import numpy as np

# Pastikan matriks CSR dan beri bobot (alpha)
alpha = 40
train_weighted = (train_mat * alpha).astype("double").tocsr()
val_mat = val_mat.tocsr()

print(f"\n✅ Train matrix shape: {train_weighted.shape}")
print(f"✅ Val matrix shape  : {val_mat.shape}")
print(f"✅ Matrix weighted with alpha={alpha}")

# Training ALS (PAKSA PAKAI CPU agar tidak error "No CUDA extension ...")
print("\n🚀 Training ALS model (CPU mode, use_gpu=False) ...")
als_model = AlternatingLeastSquares(
    factors=128,
    regularization=0.05,
    iterations=20,
    num_threads=8,
    use_gpu=False   # <- PENTING: paksa CPU untuk menghindari error CUDA
)


# NOTE: penting -> fit dengan matriks berorientasi (users x items) tanpa .T
als_model.fit(train_weighted)

# Sinkronisasi dimensi (cek cepat)
n_users, n_items = train_weighted.shape
print(f"📊 Model factors shapes: user_factors={als_model.user_factors.shape}, item_factors={als_model.item_factors.shape}")
print(f"📊 Expect: {n_users} users, {n_items} items")

# Generate rekomendasi per-user (pakai satu baris user_items setiap iterasi)
print("\n🎁 Membuat rekomendasi untuk tiap user ...")
als_preds = []
for u in tqdm(range(n_users)):
    # ambil single-row CSR untuk user u
    user_row = train_weighted[u]            # shape (1, n_items)
    recs, _ = als_model.recommend(
        userid=u,
        user_items=user_row,
        N=10,
        filter_already_liked_items=True
    )
    als_preds.append(list(map(int, recs)))  # pastikan int list

# Siapkan ground-truth dari val_data
val_actual = [
    val_data[val_data['user_idx'] == u]['item_idx'].tolist()
    for u in range(n_users)
]

# Fungsi APK / MAPK (sama seperti yang kamu punya)
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k) if actual else 0.0

def mapk(actual_list, predicted_list, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual_list, predicted_list)])

map10 = mapk(val_actual, als_preds, k=10)
print(f"\n🎯 MAP@10 (ALS, CPU mode): {map10:.4f}")

# Buat mapping inverse (kalau belum ada)
item_inv_map = {v: k for k, v in item_map.items()}
user_inv_map = {v: k for k, v in user_map.items()}

# --- Buat dataframe hasil rekomendasi ---
als_df = pd.DataFrame({
    'user_id': [user_inv_map[u] for u in range(n_users)],
    **{f'item_{i+1}': [
        item_inv_map[recs[i]] if len(recs) > i else None for recs in als_preds
    ] for i in range(10)}
})

display(als_df.head())

# Simpan ke CSV
als_df.to_csv("als_recommendations.csv", index=False)
print("💾 Hasil rekomendasi disimpan ke: als_recommendations.csv")



✅ Train matrix shape: (13876, 123069)
✅ Val matrix shape  : (13876, 123069)
✅ Matrix weighted with alpha=40

🚀 Training ALS model (CPU mode, use_gpu=False) ...


  0%|          | 0/20 [00:00<?, ?it/s]

📊 Model factors shapes: user_factors=(13876, 128), item_factors=(123069, 128)
📊 Expect: 13876 users, 123069 items

🎁 Membuat rekomendasi untuk tiap user ...


100%|██████████| 13876/13876 [00:49<00:00, 282.73it/s]



🎯 MAP@10 (ALS, CPU mode): 0.0180


,user_id,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10
0,8,0440212561,0345459202,038082101X,0452282152,0316601950,0316782505,0060959037,0743225406,0440217563,044021145X
1,99,0385505833,0743418174,0375414053,0385508042,0671568175,0671021001,0385335482,0553279912,0446610399,038550120X
2,114,0440235162,0446611867,044023722X,0515135062,0440241537,0440236673,0446613436,0345450728,0743411250,0553579983
3,160,0671041789,0060976845,0515103292,0345459202,0670896764,0446532231,0060938455,0684801051,0375504613,0395927218
4,183,067976402X,0312278586,0446360856,014028009X,0452282152,0451198638,0425172546,034538475X,0345342968,0060987529


💾 Hasil rekomendasi disimpan ke: als_recommendations.csv


In [10]:
# 🔷 VERSI Tuning — MODEL-BASED RECOMMENDER (ALS Optimized & Fixed + MAP@10)
from implicit.als import AlternatingLeastSquares
from tqdm import tqdm
import numpy as np
import pandas as pd

# --- 1️⃣ Tentukan range hyperparameter ---
factors_list = [64, 128]          # jumlah latent factors
regularization_list = [0.05, 0.1] # regularisasi
iterations_list = [15, 20]        # jumlah iterasi
alpha_list = [20, 40]             # weighting user-item matrix

best_map10 = -1
best_params = {}
best_preds = None

# --- 2️⃣ Fungsi MAP@10 ---
def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k) if actual else 0.0

def mapk(actual_list, predicted_list, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual_list, predicted_list)])

# --- 3️⃣ Mulai grid search tuning ---
for factors in factors_list:
    for reg in regularization_list:
        for iter_num in iterations_list:
            for alpha in alpha_list:
                # Bobot matrix
                train_weighted = (train_mat * alpha).astype("double").tocsr()
                val_mat_csr = val_mat.tocsr()

                # Inisialisasi model ALS
                als_model = AlternatingLeastSquares(
                    factors=factors,
                    regularization=reg,
                    iterations=iter_num,
                    num_threads=8,
                    use_gpu=False
                )
                
                # Fit ALS
                als_model.fit(train_weighted)

                # Generate rekomendasi untuk tiap user
                als_preds = []
                for u in range(n_users):
                    user_row = train_weighted[u]
                    recs, _ = als_model.recommend(
                        userid=u,
                        user_items=user_row,
                        N=10,
                        filter_already_liked_items=True
                    )
                    als_preds.append(list(map(int, recs)))

                # Ground truth
                val_actual = [
                    val_data[val_data['user_idx'] == u]['item_idx'].tolist()
                    for u in range(n_users)
                ]

                # Hitung MAP@10
                map10_score = mapk(val_actual, als_preds, k=10)
                print(f"factors={factors}, reg={reg}, iter={iter_num}, alpha={alpha} => MAP@10={map10_score:.4f}")

                # Simpan kombinasi terbaik
                if map10_score > best_map10:
                    best_map10 = map10_score
                    best_params = {"factors": factors, "regularization": reg, "iterations": iter_num, "alpha": alpha}
                    best_preds = als_preds.copy()

# --- 4️⃣ Tampilkan hasil terbaik ---
print("\n✅ Best hyperparameters:")
print(best_params)
print(f"🎯 Best MAP@10: {best_map10:.4f}")

# --- 5️⃣ Buat DataFrame hasil rekomendasi ---
item_inv_map = {v: k for k, v in item_map.items()}
user_inv_map = {v: k for k, v in user_map.items()}

als_df = pd.DataFrame({
    'user_id': [user_inv_map[u] for u in range(n_users)],
    **{f'item_{i+1}': [
        item_inv_map[recs[i]] if len(recs) > i else None for recs in best_preds
    ] for i in range(10)}
})

display(als_df.head())
als_df.to_csv("als_recommendations_tuned.csv", index=False)
print("💾 Hasil rekomendasi tuning disimpan ke: als_recommendations_tuned.csv")


  0%|          | 0/15 [00:00<?, ?it/s]

factors=64, reg=0.05, iter=15, alpha=20 => MAP@10=0.0149


  0%|          | 0/15 [00:00<?, ?it/s]

factors=64, reg=0.05, iter=15, alpha=40 => MAP@10=0.0145


  0%|          | 0/20 [00:00<?, ?it/s]

...

factors=128, reg=0.1, iter=20, alpha=20 => MAP@10=0.0179


  0%|          | 0/20 [00:00<?, ?it/s]

factors=128, reg=0.1, iter=20, alpha=40 => MAP@10=0.0184

✅ Best hyperparameters:
{'factors': 128, 'regularization': 0.1, 'iterations': 20, 'alpha': 40}
🎯 Best MAP@10: 0.0184


,user_id,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10
0,8,0316601950,038082101X,0345459202,0684848783,0375727345,0743225406,0385720106,0749397543,038079487X,0440212561
1,99,0671021001,0446610399,0385505833,038550120X,0385335482,0743418174,0312966091,0385472951,0062502182,0028604199
2,114,0446611867,0061015725,0515135062,0553576801,044661162X,034541389X,0440235162,0553272837,0446611212,0553578685
3,160,0446605239,0156027321,0439139600,0140067477,043935806X,0385504209,0439064872,0446611212,0060977493,059035342X
4,183,059035342X,0451456521,0142001740,0399151648,068483376X,0671042572,0316284955,0312282990,0060936363,0679751521


💾 Hasil rekomendasi tuning disimpan ke: als_recommendations_tuned.csv


In [12]:
# 🔷 VERSI FineTuning — MODEL-BASED RECOMMENDER (ALS Optimized & Fixed + MAP@10)
from implicit.als import AlternatingLeastSquares
from tqdm import tqdm
import numpy as np
import pandas as pd

# --- 1️⃣ Siapkan matrix weighted (alpha) ---
alpha = 40
train_weighted = (train_mat * alpha).astype("double").tocsr()
val_mat_csr = val_mat.tocsr()

# --- 2️⃣ Misal model ALS pretrained (dari training sebelumnya) ---
# Kalau belum ada, kita bisa latihan dulu 5 iterasi untuk “pretrain”
pretrain_model = AlternatingLeastSquares(
    factors=128,
    regularization=0.05,
    iterations=5,    # iterasi sedikit untuk pretrain
    num_threads=8,
    use_gpu=False
)
pretrain_model.fit(train_weighted)
print("✅ Pretrained ALS selesai!")

# --- 3️⃣ Finetuning ---
# Kita ambil bobot dari pretrain_model
finetune_model = AlternatingLeastSquares(
    factors=128,
    regularization=0.05,
    iterations=10,   # iterasi tambahan untuk finetuning
    num_threads=8,
    use_gpu=False
)
# Copy bobot dari pretrain_model
finetune_model.user_factors = pretrain_model.user_factors.copy()
finetune_model.item_factors = pretrain_model.item_factors.copy()

# Latih lagi dengan dataset yang sama atau subset baru
finetune_model.fit(train_weighted)
print("🎯 Finetuning ALS selesai!")

# --- 4️⃣ Generate rekomendasi per user ---
n_users, n_items = train_weighted.shape
als_preds = []
for u in tqdm(range(n_users)):
    user_row = train_weighted[u]
    recs, _ = finetune_model.recommend(
        userid=u,
        user_items=user_row,
        N=10,
        filter_already_liked_items=True
    )
    als_preds.append(list(map(int, recs)))

# --- 5️⃣ Evaluasi MAP@10 ---
val_actual = [
    val_data[val_data['user_idx'] == u]['item_idx'].tolist()
    for u in range(n_users)
]

def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k) if actual else 0.0

def mapk(actual_list, predicted_list, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual_list, predicted_list)])

map10 = mapk(val_actual, als_preds, k=10)
print(f"\n📊 MAP@10 (ALS Finetuned): {map10:.4f}")

# --- 6️⃣ Buat DataFrame hasil rekomendasi ---
item_inv_map = {v: k for k, v in item_map.items()}
user_inv_map = {v: k for k, v in user_map.items()}

als_df = pd.DataFrame({
    'user_id': [user_inv_map[u] for u in range(n_users)],
    **{f'item_{i+1}': [
        item_inv_map[recs[i]] if len(recs) > i else None for recs in als_preds
    ] for i in range(10)}
})

display(als_df.head())
als_df.to_csv("als_recommendations_finetuned.csv", index=False)
print("💾 Hasil rekomendasi finetuning disimpan ke: als_recommendations_finetuned.csv")


  0%|          | 0/5 [00:00<?, ?it/s]

✅ Pretrained ALS selesai!


  0%|          | 0/10 [00:00<?, ?it/s]

🎯 Finetuning ALS selesai!


100%|██████████| 13876/13876 [00:48<00:00, 284.68it/s]



📊 MAP@10 (ALS Finetuned): 0.0180


,user_id,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10
0,8,0316601950,0684848783,0345459202,0440212561,038082101X,0679781587,0375706771,068483376X,0671003755,0020811853
1,99,0385335482,0385505833,038550120X,0312983824,0743418174,0385497466,0385508042,0399148639,0385484518,0385503822
2,114,0440224764,0515135062,0446611867,042518286X,0553578316,0425190641,0743411250,0440225825,0451206525,0440222656
3,160,0060930535,0451206525,0385265700,0345378482,002542730X,067088300X,0451167805,039592720X,0875421180,0451169530
4,183,0440226430,0316284955,0375719180,0345413903,0446613266,0380807343,0446610038,0446530077,0345339703,0451180232


💾 Hasil rekomendasi finetuning disimpan ke: als_recommendations_finetuned.csv
